In [1]:
import torch
import torch.nn as nn

In [2]:
input_feat = torch.tensor([[4, 1, 7, 5], [4, 4, 2, 5], [7, 7, 2, 4], [1, 0, 2, 4]], dtype=torch.float32).unsqueeze(0).unsqueeze(0)

# 创建2*2卷积 通道数1 输出通道数1 padding为same
conv2d = nn.Conv2d(1,1,(2,2),stride=1,padding='same',bias=False)
conv2d.weight,conv2d.bias

(Parameter containing:
 tensor([[[[-0.1733,  0.4964],
           [-0.4237, -0.2133]]]], requires_grad=True),
 None)

In [3]:
# 卷积核要有四个维度(输入通道数，输出通道数，高，宽)
kernels = torch.tensor([[[[1,0],[2,1]]]], dtype=torch.float32)
conv2d.weight = nn.Parameter(kernels, requires_grad=False)
conv2d.weight,conv2d.bias

(Parameter containing:
 tensor([[[[1., 0.],
           [2., 1.]]]]),
 None)

In [4]:
output = conv2d(input_feat)
output

/usr/local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1041.)
  return F.conv2d(input, weight, bias, self.stride,


tensor([[[[16., 11., 16., 15.],
          [25., 20., 10., 13.],
          [ 9.,  9., 10., 12.],
          [ 1.,  0.,  2.,  4.]]]])

##### 深度可分离卷积

由 Depthwise（DW）和 Pointwise（PW）这两部分卷积组合而成的。


In [8]:
x = torch.rand((3,5,5)).unsqueeze(0)
x,x.shape

(tensor([[[[0.7369, 0.9071, 0.2719, 0.7429, 0.5879],
           [0.1192, 0.5065, 0.5980, 0.2418, 0.8072],
           [0.2732, 0.2270, 0.0095, 0.3047, 0.7819],
           [0.1943, 0.5149, 0.8443, 1.0000, 0.2291],
           [0.6862, 0.0278, 0.7733, 0.3798, 0.4050]],
 
          [[0.6785, 0.1136, 0.8473, 0.6956, 0.5490],
           [0.6566, 0.2725, 0.0878, 0.8746, 0.6738],
           [0.4568, 0.5777, 0.3748, 0.4598, 0.8947],
           [0.9329, 0.5418, 0.3544, 0.5471, 0.8430],
           [0.7872, 0.1279, 0.3041, 0.3642, 0.1038]],
 
          [[0.8302, 0.6458, 0.5462, 0.9407, 0.3037],
           [0.6600, 0.8931, 0.1972, 0.9240, 0.3726],
           [0.6341, 0.9038, 0.0424, 0.5048, 0.0781],
           [0.1096, 0.0087, 0.2419, 0.5548, 0.0718],
           [0.1962, 0.9521, 0.2223, 0.9961, 0.3262]]]]),
 torch.Size([1, 3, 5, 5]))

In [16]:
# dw卷积实现
in_channels_dw = x.shape[1]
out_channels_dw = x.shape[1]

kernel_size = 3
stride = 1

dw = nn.Conv2d(in_channels_dw,out_channels_dw, kernel_size, stride, groups=in_channels_dw)

In [17]:
# pw卷积实现
in_channels_dw = out_channels_dw
out_channels_dw = 4
kernel_size_pw = 1
pw = nn.Conv2d(in_channels_dw,out_channels_dw,kernel_size_pw,stride)
out = pw(dw(x))
out.shape,out

(torch.Size([1, 4, 3, 3]),
 tensor([[[[ 0.3413,  0.3161,  0.3254],
           [ 0.3296,  0.2423,  0.1546],
           [ 0.3787,  0.2530,  0.3206]],
 
          [[ 0.0482,  0.0014,  0.0105],
           [ 0.0224, -0.0665, -0.1549],
           [ 0.0889, -0.0814,  0.0283]],
 
          [[ 0.5765,  0.4159,  0.5091],
           [ 0.4036,  0.5277,  0.7693],
           [ 0.5647,  0.6013,  0.3638]],
 
          [[ 0.4089,  0.1677,  0.2641],
           [ 0.1844,  0.2763,  0.4889],
           [ 0.4197,  0.2695,  0.1902]]]], grad_fn=<ConvolutionBackward0>))